In [6]:
# coding:utf-8
import arrow
import qgrid
import seaborn as sns
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

import matplotlib.pyplot as plt
import jieba
    

from sklearn.model_selection import train_test_split


%matplotlib inline

sns.set(style='whitegrid')

In [7]:
trans_prov_allcount = pd.read_csv('../result_data/article_predict_result.csv',encoding='utf-8',index_col = False, delimiter=',' )
trans_prov_allcount.head()

,code,company,label_property,title,url,article
0,HK0001,长和,中性,赫斯基首季盈利增32%胜预期,https://news.10jqka.com.cn/m611120552/,"<p>【财华社讯】<a data-code=""HK0001"" data-marketid=""..."
1,HK0001,长和,中性,李泽钜接手后的屈臣氏：前有丝芙兰后有KOL 腾讯阿里争买股份 谁才是最大的赢家,https://m.jiemian.com/article/3002332.html,<p><span>图品来源：视觉中国</span></p><p>文 | 财联社 万千</p>...
2,HK0001,长和,中性,东兴证券2019年4月十大金股,https://mp.weixin.qq.com/s?__biz=MzA3NTA2NTUwO...,"<p class=""profile_meta"">\n<label class=""profil..."
3,HK0001,长和,中性,李泽钜接棒一年 长和系继续全球收购,https://news.10jqka.com.cn/m610461495/,<p>时代周报记者蔡颖发自广州</p><p>距“超人”李嘉诚退休已经快一年。在这一年的时间里...
4,HK0001,长和,利好,杰富瑞：若淡马锡售10%屈臣氏持股 长和净资产将升15%,https://news.10jqka.com.cn/m610409446/,"<p>杰富瑞：若淡马锡成功出售10%屈臣氏持股，<a data-code=""HK0001"" ..."


In [8]:
jieba.load_userdict("../corpus/Organization-Names-Corpus.txt")
jieba.load_userdict("../corpus/short-company.txt")
jieba.load_userdict("../corpus/THUOCL_caijing.txt")
jieba.load_userdict("../corpus/company_dict.txt")
jieba.load_userdict("../corpus/username.txt")


In [9]:
def stopwordslist(filepath):  
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]  
    return stopwords  
  
def seg_sentence(sentence):  
    outstr = []
    sentence_seged = jieba.cut(sentence.strip())  
    stopwords = stopwordslist('../corpus/stopword.txt')  # 这里加载停用词的路径  
    for word in sentence_seged:  
        if word not in stopwords:  
            if word != '\t':  
                outstr.append(word)   
    return outstr 
def clearstr(line_str):
    rule = re.compile(r"[^\u4e00-\u9fa5]")
    line_str = rule.sub('',line_str)
    rule = re.compile(u"[^\u4e00-\u9fa5]")
    line_str = rule.sub('',line_str)
    return re.sub("[\d+\s+\.\!\/_,$%^*(+\"\'<>=【】\-]+|[+——！，。？、~@#￥%……&*（）：]+", "",line_str)



In [10]:
import re
trans_prov_allcount['title'] =trans_prov_allcount['title'].apply(clearstr)
trans_prov_allcount['article'] =trans_prov_allcount['article'].apply(clearstr)
trans_prov_allcount.head()

,code,company,label_property,title,url,article
0,HK0001,长和,中性,赫斯基首季盈利增胜预期,https://news.10jqka.com.cn/m611120552/,财华社讯长和旗下加拿大赫斯基能源公布首季业绩盈利亿元加元下同同比上升扣除特殊项目后每股盈利元...
1,HK0001,长和,中性,李泽钜接手后的屈臣氏前有丝芙兰后有腾讯阿里争买股份谁才是最大的赢家,https://m.jiemian.com/article/3002332.html,图品来源视觉中国文财联社万千去年月李嘉诚退休李泽钜正式接管父亲的万亿商业帝国同时接手了旗下已...
2,HK0001,长和,中性,东兴证券年月十大金股,https://mp.weixin.qq.com/s?__biz=MzA3NTA2NTUwO...,功能介绍推送东兴研究所的研究成果来服务机构投资者外资风险偏好扰动放大市场情绪波动随着减税等一...
3,HK0001,长和,中性,李泽钜接棒一年长和系继续全球收购,https://news.10jqka.com.cn/m610461495/,时代周报记者蔡颖发自广州距超人李嘉诚退休已经快一年在这一年的时间里外界好奇新掌门人李泽钜秉承...
4,HK0001,长和,利好,杰富瑞若淡马锡售屈臣氏持股长和净资产将升,https://news.10jqka.com.cn/m610409446/,杰富瑞若淡马锡成功出售屈臣氏持股长和的净资产值将可提升


In [11]:
trans_prov_allcount['title_words'] =trans_prov_allcount['title'].apply(seg_sentence)
trans_prov_allcount.head()

,code,company,label_property,title,url,article,title_words
0,HK0001,长和,中性,赫斯基首季盈利增胜预期,https://news.10jqka.com.cn/m611120552/,财华社讯长和旗下加拿大赫斯基能源公布首季业绩盈利亿元加元下同同比上升扣除特殊项目后每股盈利元...,"[赫斯基, 首季, 盈利, 增胜, 预期]"
1,HK0001,长和,中性,李泽钜接手后的屈臣氏前有丝芙兰后有腾讯阿里争买股份谁才是最大的赢家,https://m.jiemian.com/article/3002332.html,图品来源视觉中国文财联社万千去年月李嘉诚退休李泽钜正式接管父亲的万亿商业帝国同时接手了旗下已...,"[李, 泽钜, 接手, 屈臣氏, 前有, 丝芙兰, 腾讯, 阿里, 争买, 股份, 最大, 赢家]"
2,HK0001,长和,中性,东兴证券年月十大金股,https://mp.weixin.qq.com/s?__biz=MzA3NTA2NTUwO...,功能介绍推送东兴研究所的研究成果来服务机构投资者外资风险偏好扰动放大市场情绪波动随着减税等一...,"[东兴证券, 年, 月, 十大, 金股]"
3,HK0001,长和,中性,李泽钜接棒一年长和系继续全球收购,https://news.10jqka.com.cn/m610461495/,时代周报记者蔡颖发自广州距超人李嘉诚退休已经快一年在这一年的时间里外界好奇新掌门人李泽钜秉承...,"[李, 泽钜, 接棒, 一年, 长, 和系, 继续, 全球, 收购]"
4,HK0001,长和,利好,杰富瑞若淡马锡售屈臣氏持股长和净资产将升,https://news.10jqka.com.cn/m610409446/,杰富瑞若淡马锡成功出售屈臣氏持股长和的净资产值将可提升,"[杰富瑞, 淡马锡, 售, 屈臣氏, 持股, 长和, 净资产, 将升]"


In [12]:
trans_prov_allcount['title_words'].values[0][0]

'赫斯基'

In [13]:
cate_dic = {'利好':1, '中性':2, '利空':3}


def preprocess_text(row):
    cate_dic = {'利好':1, '中性':2, '利空':3}
    cate = row['label_property']
    result_str = "__label__"+str(cate_dic[cate])+" , "+" ".join(row['title_words'])
    return result_str

In [14]:
trans_prov_allcount['title_train'] = trans_prov_allcount.apply(preprocess_text, axis=1)

In [15]:
sentences = []

for sentence in trans_prov_allcount['title_train'].values:
    sentences.append(sentence)
sentences[3]

'__label__2 , 李 泽钜 接棒 一年 长 和系 继续 全球 收购'

In [17]:
with open('train_data.txt', 'w') as f:
    for item in sentences:
        f.write("%s\n" % item)

In [18]:
import fasttext
classifier = fasttext.supervised('train_data.txt', 'classifier.model', label_prefix='__label__')



In [19]:

result = classifier.test('train_data.txt')
print('P@1:'+str(result.precision))
print('R@1:'+str(result.recall))
print('Number of examples:'+str(result.nexamples))

P@1:0.8453095154824998
R@1:0.8453095154824998
Number of examples:37171


In [31]:
str_cln= "两市地位震荡沪指险守2900点 ST板块抛跌停潮"
str_cln= clearstr(str_cln)
str_arr = seg_sentence(str_cln)

str_arr = [" ".join(str_arr)]
str_arr

['两市 地位 震荡 沪 指险 守点 板块 抛 跌停 潮']

In [32]:
label_to_cate = {1:'利好', 2:'中性', 3:'利空'}

labels = classifier.predict(str_arr)

print(labels)
print(label_to_cate[int(labels[0][0])])

[['3']]
利空


In [37]:
str_cln= "连年亏损 你为什么还要买入美团点评"
str_cln= clearstr(str_cln)
str_arr = seg_sentence(str_cln)

str_arr = [" ".join(str_arr)]
str_arr

['连年 亏损 买入 美团点评']

In [38]:
label_to_cate = {1:'利好', 2:'中性', 3:'利空'}

labels = classifier.predict(str_arr)

print(labels)
print(label_to_cate[int(labels[0][0])])

[['3']]
利空
